In [2]:
import pandas as pd
import json
from datetime import datetime

In [3]:
# Function to read a json data file
def load_json(file_path):
    try:
        with open (file_path, 'r') as file:
            
            data = json.load(file)
            return data
        
    except FileNotFoundError:
        
        print(f"Error: File {file_path} not found.")
        return None
    
    except json.JSONDecodeError:
        
        print(f"Invalid Json format.")
        return None
    

In [7]:
#checking if function is working well or not
match_data = load_json(r'C:\Users\yoges\OneDrive\Documents\Programming\Proj-X\match.json')

 check the various types of record returned by the function

In [8]:
segment_types = set(segment['type'] for segment in match_data['data']['segments'])
segment_types

{'player-loadout',
 'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [9]:
df_lst = []

In [ ]:
match

In [ ]:
# as data is more complex to read we need to make a function to process this data

def process_match_data(data):
    if not data or 'data' not in data:
        return None, None, None
    
    match_data = data['data']
    metadata = match_data['metadata']
    segments = match_data['segments']
    
    #match summary
    
    match_summary = {
        'match_id': match_data['attributes']['id'],
        'map_name':metadata['mapName'],
        'mode': metadata['modeName'],
        'date_started': datetime.strptime(metadata['dateStarted'], '%Y-%m-%d%H:%M:%S.%f%z'),
        'duration_minutes': metadata['duration']/1000/60, #here converting ms to secounds
        'rounds_played': metadata['rounds'],
        'is_ranked': metadata['isRanked']
    }
    match_df = pd.DataFrame([match_summary])
    
    #Creating a team summary DataFrame
    
    team_data = []
    for segment in segments:
        if segment['type'] == 'team-summary':
            team_data.append({
                'team_id': segment['attributes']['teamId'],
                'name': segment['metadata']['name'],
                'has_won': segment['metadata']['hasWon'],
                'rounds_won': segment['stats']['roundsWon']['value'],
                'rounds_lost': segment['stats']['roundsLost']['value'],
                'score': segment['stats']['score']['value'],
                'kills': segment['stats']['kills']['value'],
                'deaths': segment['stats']['deaths']['value'],
                'assists': segment['stats']['assists']['value'],
                'damage': segment['stats']['damage']['value']
            })
    team_df = pd.DataFrame(team_data)
    
    